## 1. Create the compute instance

In [4]:
# import required libraries
import os
from azure.ai.ml import MLClient
from azure.ai.ml.entities import WorkspaceConnection
# Import required libraries
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

# Import required libraries
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

from azure.ai.ml.entities import ComputeInstance 
from azure.ai.ml.entities import CustomApplications, ImageSettings, EndpointsSettings, VolumeSettings 

ml_client = MLClient.from_config(credential=credential)

Found the config file in: C:\code\contoso-chat\config.json


In [5]:
# create custom application from default promptflow image
image = ImageSettings(reference='mcr.microsoft.com/azureml/promptflow/promptflow-runtime-stable:latest') 
endpoints = [EndpointsSettings(published=8081, target=8080)]
app = CustomApplications(name='promptflow-runtime',endpoints=endpoints,bind_mounts=[],image=image,environment_variables={}) 

In [6]:
# create compute instance for prompt flow
ci_basic_name = "contoso-chat-ci"
ci_basic = ComputeInstance(name=ci_basic_name, size="STANDARD_DS12_V2",custom_applications=[app]) 
ml_client.begin_create_or_update(ci_basic)

In [9]:
# List compute instances
for ci in ml_client.compute.list():
    print(ci.name)

contoso-support
contoso-chat-ci


## 2. Create the prompt flow runtime

Once the compute instance is created. [Follow these instructions to create a runtime in AI Studio.](https://learn.microsoft.com/en-us/azure/ai-studio/how-to/create-manage-runtime)

## 3. Run the prompt flow on the runtime to install the dependencies

In [39]:
from promptflow import PFClient
pf_client = PFClient()

In [37]:
# Add a question to test the base prompt flow.
output = pf_client.test(
    flow=".", # Path to the flow directory
    inputs={},
)
output['version']

2023-12-02 16:28:53 -0600   32284 execution.flow     INFO     Start to run 1 nodes with concurrency level 16.
2023-12-02 16:28:53 -0600   32284 execution.flow     INFO     Executing node add_custom_packages. node run id: 61196823-36e7-46d5-8aef-a316938213b3_add_custom_packages_0
2023-12-02 16:28:58 -0600   32284 execution.flow     INFO     Node add_custom_packages completes.


In [42]:
config_path = "../config.json"
from promptflow.azure import PFClient
pf_azure_client = PFClient.from_config(credential=credential, path=config_path)

Found the config file in: ..\config.json


In [53]:
# Update the runtime to the name of the runtime you created previously
runtime = "promptflow-contoso-chat-rt"
# load flow
flow = "."
run_name="update_runtime_test"
data = "../data/emptydata.jsonl"

In [54]:
base_run = pf_azure_client.run(
    flow=flow,
    data=data,
    runtime=runtime,
    # create a display name as current datetime
    display_name=run_name,
    name=run_name
)
print(base_run)

Portal url: https://ml.azure.com/prompts/flow/bulkrun/run/update_runtime_test/details?wsid=/subscriptions/91d27443-f037-45d9-bb0c-428256992df6/resourceGroups/rg-contosooutdoor/providers/Microsoft.MachineLearningServices/workspaces/contoso-outdoor-chat
name: update_runtime_test
created_on: '2023-12-02T22:43:21.857948+00:00'
status: Preparing
display_name: update_runtime_test
description: null
tags: {}
properties:
  azureml.promptflow.runtime_name: promptflow-contoso-chat-rt
  azureml.promptflow.runtime_version: 20231114.v2
  azureml.promptflow.definition_file_name: flow.dag.yaml
  azureml.promptflow.session_id: a1b7e6e546da59083e94b2d93551ca31361ace3ac27323ba
  azureml.promptflow.flow_lineage_id: 87d042a9621bb7af861108edbde710be6755dfe18b662e1352c9f583bea27d39
  azureml.promptflow.flow_definition_datastore_name: workspaceblobstore
  azureml.promptflow.flow_definition_blob_path: LocalUpload/b1d8062cfe3aa0bab141a7f257c219b4/runtime/flow.dag.yaml
  _azureml.evaluation_run: promptflow.Batch

In [55]:
pf_azure_client.stream(base_run)

2023-12-02 22:43:27 +0000      35 promptflow-runtime INFO     [update_runtime_test] Receiving v2 bulk run request 678004c7-044e-4958-af4e-706af83d6602: {"flow_id": "update_runtime_test", "flow_run_id": "update_runtime_test", "flow_source": {"flow_source_type": 1, "flow_source_info": {"snapshot_id": "c9e42b40-0d3d-43c9-9f62-ba733c452a29"}, "flow_dag_file": "flow.dag.yaml"}, "log_path": "https://stcontosoout699609817029.blob.core.windows.net/3a32a40b-674b-4e11-8e69-4d28b93d4235-azureml/ExperimentRun/dcid.update_runtime_test/logs/azureml/executionlogs.txt?sv=2019-07-07&sr=b&sig=**data_scrubbed**&skoid=e600d0b5-5a77-431c-ba8a-d6cf970648ba&sktid=72f988bf-86f1-41af-91ab-2d7cd011db47&skt=2023-12-02T21%3A15%3A12Z&ske=2023-12-04T05%3A25%3A12Z&sks=b&skv=2019-07-07&st=2023-12-02T22%3A33%3A26Z&se=2023-12-03T06%3A43%3A26Z&sp=rcw", "app_insights_instrumentation_key": "InstrumentationKey=**data_scrubbed**;IngestionEndpoint=https://westus2-4.in.applicationinsights.azure.com/;LiveEndpoint=https://westu

In [56]:
details = pf_azure_client.get_details(base_run)

,inputs.line_number,outputs.version
outputs.line_number,,
0,0,"[Package, ------------------------------------..."


In [58]:
details["outputs.version"][0]

['Package',
 '------------------------------------',
 'adal',
 'aiofiles',
 'aiohttp',
 'aiosignal',
 'alembic',
 'antlr4-python3-runtime',
 'argcomplete',
 'asgiref',
 'async-timeout',
 'attrs',
 'azure-ai-ml',
 'azure-common',
 'azure-core',
 'azure-core-tracing-opentelemetry',
 'azure-cosmos',
 'azure-data-tables',
 'azure-graphrbac',
 'azure-identity',
 'azure-keyvault',
 'azure-keyvault-certificates',
 'azure-keyvault-keys',
 'azure-keyvault-secrets',
 'azure-mgmt-authorization',
 'azure-mgmt-containerregistry',
 'azure-mgmt-core',
 'azure-mgmt-keyvault',
 'azure-mgmt-resource',
 'azure-mgmt-storage',
 'azure-monitor-opentelemetry',
 'azure-monitor-opentelemetry-exporter',
 'azure-search-documents',
 'azure-storage-blob',
 'azure-storage-file-datalake',
 'azure-storage-file-share',
 'azureml-ai-monitoring',
 'azureml-core',
 'azureml-dataprep',
 'azureml-dataprep-native',
 'azureml-dataprep-rslex',
 'azureml-mlflow',
 'backports.tempfile',
 'backports.weakref',
 'banal',
 'bcrypt'